In [4]:
import requests
from urllib.parse import quote
import pandas as pd

- api가져오기

In [5]:
with open('kakaoapikey.txt') as file:
    kakao_key=file.read()
    

In [6]:
len(kakao_key)

32

- url 만들기

In [7]:
local_url = "https://dapi.kakao.com/v2/local/search/address.json"
addr = '제주특별자치도 제주시 조천읍 비자림로 768'
url = f'{local_url}?query={quote(addr)}'
url

'https://dapi.kakao.com/v2/local/search/address.json?query=%EC%A0%9C%EC%A3%BC%ED%8A%B9%EB%B3%84%EC%9E%90%EC%B9%98%EB%8F%84%20%EC%A0%9C%EC%A3%BC%EC%8B%9C%20%EC%A1%B0%EC%B2%9C%EC%9D%8D%20%EB%B9%84%EC%9E%90%EB%A6%BC%EB%A1%9C%20768'

In [8]:
# Header : "Authorization: KakaoAK {REST_API_KEY}"
header = {"Authorization": f'KakaoAK {kakao_key}'}

In [9]:
result = requests.get(url, headers=header).json()
result

{'documents': [{'address': {'address_name': '제주특별자치도 제주시 조천읍 교래리 342-2',
    'b_code': '5011025929',
    'h_code': '5011025900',
    'main_address_no': '342',
    'mountain_yn': 'N',
    'region_1depth_name': '제주특별자치도',
    'region_2depth_name': '제주시',
    'region_3depth_h_name': '조천읍',
    'region_3depth_name': '조천읍 교래리',
    'sub_address_no': '2',
    'x': '126.688504038171',
    'y': '33.4348735442325'},
   'address_name': '제주특별자치도 제주시 조천읍 비자림로 768',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '제주특별자치도 제주시 조천읍 비자림로 768',
    'building_name': '',
    'main_building_no': '768',
    'region_1depth_name': '제주특별자치도',
    'region_2depth_name': '제주시',
    'region_3depth_name': '조천읍 교래리',
    'road_name': '비자림로',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '126.688504038171',
    'y': '33.4348735442325',
    'zone_no': '63346'},
   'x': '126.688504038171',
   'y': '33.4348735442325'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1

In [21]:
lng=float(result['documents'][0]['x'])
lat=float(result['documents'][0]['y'])
lng, lat

(126.688504038171, 33.4348735442325)

- 제주명소 데이터프레임에 위도 경도 넣기

In [22]:
df = pd.read_csv('제주명소.csv')
df

,명소,도로명주소
0,산굼부리,제주특별자치도 제주시 조천읍 비자림로 768
1,거문오름,제주특별자치도 제주시 조천읍 거문오름길 5
2,송악산,제주특별자치도 서귀포시 대정읍 형제해안로 318
3,섭지코지,제주특별자치도 서귀포시 성산읍 섭지코지로 93-66
4,만장굴,제주특별자치도 제주시 구좌읍 만장굴길 182
5,성산일출봉,제주특별자치도 서귀포시 성산읍 일주동로 4282
6,오설록,제주특별자치도 서귀포시 안덕면 신화역사로 15
7,주상절리,제주특별자치도 서귀포시 이어도로 36-30(중문동)
8,천제연폭포,제주특별자치도 서귀포시 천제연로 132(중문동)
9,제주관덕정,제주특별자치도 제주시 관덕로 19(삼도이동)


In [23]:
df.index

RangeIndex(start=0, stop=10, step=1)

In [24]:
lng_list, lat_list =[],[]
for i in df.index:
    url = f'{local_url}?query={quote(df.도로명주소[i])}'
    result = requests.get(url, headers=header).json()
    lng=float(result['documents'][0]['x'])
    lat=float(result['documents'][0]['y'])
    lng_list.append(lng)
    lat_list.append(lat)
    

In [25]:
df['위도'] = lat_list
df['경도'] = lng_list
df

,명소,도로명주소,위도,경도
0,산굼부리,제주특별자치도 제주시 조천읍 비자림로 768,33.434874,126.688504
1,거문오름,제주특별자치도 제주시 조천읍 거문오름길 5,33.459582,126.712604
2,송악산,제주특별자치도 서귀포시 대정읍 형제해안로 318,33.206931,126.290224
3,섭지코지,제주특별자치도 서귀포시 성산읍 섭지코지로 93-66,33.432648,126.930174
4,만장굴,제주특별자치도 제주시 구좌읍 만장굴길 182,33.528377,126.771616
5,성산일출봉,제주특별자치도 서귀포시 성산읍 일주동로 4282,33.446254,126.911561
6,오설록,제주특별자치도 서귀포시 안덕면 신화역사로 15,33.305920,126.289374
7,주상절리,제주특별자치도 서귀포시 이어도로 36-30(중문동),33.237994,126.426018
8,천제연폭포,제주특별자치도 서귀포시 천제연로 132(중문동),33.252675,126.418374
9,제주관덕정,제주특별자치도 제주시 관덕로 19(삼도이동),33.513368,126.521475


In [26]:
df.to_csv('제주명소2.csv', index=False)